# Launch import first

In [25]:
# Imports
import sys
import os

# Add the upstream directory to sys.path
upstream_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if upstream_dir not in sys.path:
    sys.path.insert(0, upstream_dir)

# Now you can import the module
from opentrons_api import ot2_api
from microtissue_manipulator import core, utils
import numpy as np 
import cv2
import time
import json
import keyboard
# from pynput import keyboard
from configs import paths
import matplotlib.pyplot as plt
import requests
import datetime
import threading
import queue
import string
import pandas as pd
import multiprocessing as mp

# Connect to Opentrons

In [2]:
openapi = ot2_api.OpentronsAPI()

### You can test you connection by toggling the lights of the robot

In [19]:
openapi.toggle_lights()

<Response [200]>

### If launched for the first time after being turned off, always home the robot:

In [20]:
openapi.home_robot()

Request status:
<Response [200]>
{
  "message": "Homing robot."
}


<Response [200]>

### If not done already, create run:

In [21]:
openapi.create_run()

Request status:
<Response [201]>
{
  "data": {
    "id": "ae7dd37c-72d5-4b45-958a-24713c6d47ff",
    "ok": true,
    "createdAt": "2025-03-21T05:39:14.826108+00:00",
    "status": "idle",
    "current": true,
    "actions": [],
    "errors": [],
    "hasEverEnteredErrorRecovery": false,
    "pipettes": [],
    "modules": [],
    "labware": [],
    "liquids": [],
    "labwareOffsets": [],
    "runTimeParameters": [],
    "outputFileIds": []
  }
}


<Response [201]>

### This command lets the robot know what kind of pipettor it has attached

In [22]:
openapi.load_pipette()

Request status:
<Response [201]>
{
  "data": {
    "id": "9deea65f-314c-480b-9f5e-1b0540943309",
    "createdAt": "2025-03-21T05:42:19.842813+00:00",
    "commandType": "loadPipette",
    "key": "9deea65f-314c-480b-9f5e-1b0540943309",
    "status": "succeeded",
    "params": {
      "pipetteName": "p300_single_gen2",
      "mount": "left"
    },
    "result": {
      "pipetteId": "a2b17333-4045-4563-85b6-fbfa9cbea1cb"
    },
    "startedAt": "2025-03-21T05:42:19.844925+00:00",
    "completedAt": "2025-03-21T05:42:22.046857+00:00",
    "intent": "setup",
    "notes": []
  }
}


<Response [201]>

# Labware declaration (Letting the robot know what there's on the deck)

### Opentrons default tiprack (black 300ul)

In [23]:
#Define a tip rack. This is the default tip rack for the robot.
TIP_RACK = "opentrons_96_tiprack_300ul"
#Load the tip rack. Slot = 1 by default.
openapi.load_labware(TIP_RACK, 11)

Labware ID:
b59d82e2-8515-44c6-8699-f5d08e735049



<Response [201]>

### VWR long tips 200ul (custom)

In [24]:
custom_labware_path = os.path.join(paths.BASE_DIR,'protocols','vwr_96_tiprack_200ul_xl.json')
with open(custom_labware_path, 'r') as json_file:
    custom_labware = json.load(json_file)

command_dict = {
            "data": custom_labware
        }

command_payload = json.dumps(command_dict)

url = openapi.get_url('runs')+ f'/{openapi.run_id}/'+ 'labware_definitions'
r = requests.post(url = url, headers = openapi.HEADERS, params = {"waitUntilComplete": True}, data = command_payload)

#------------------------------------------------------------------------------------------------------
#Define a tip rack. This is the default tip rack for the robot.
TIP_RACK = "vwr_96_tiprack_200ul_xl"
#Load the tip rack. Slot = 1 by default.
openapi.load_labware(TIP_RACK, 10, namespace='custom_beta',verbose=True)

Labware ID:
33089cc5-c152-4672-81a6-404c43e3443a



<Response [201]>

### Define a reservoir, usually a 6 well plate

In [26]:
RESERVOIR = "corning_6_wellplate_16.8ml_flat"
openapi.load_labware(RESERVOIR, 3, namespace='opentrons',verbose=True)

Labware ID:
7066508d-2b92-46d9-9b04-8e1797042ac5



<Response [201]>

### Define a well plate

In [27]:
# WELL_PLATE = "corning_24_wellplate_3.4ml_flat"
WELL_PLATE = "corning_96_wellplate_360ul_flat"
# WELL_PLATE = "corning_384_wellplate_112ul_flat"
openapi.load_labware(WELL_PLATE, 6, namespace='opentrons',verbose=True)

Labware ID:
10f32256-ffeb-426d-85f2-9ef003f1e1f6



<Response [201]>

# Pick up a tip

In [28]:
r = openapi.pick_up_tip(openapi.labware_dct['10'], "A1")

# Take off solutions

### Command that moves the pipette tip to hover above the well of the well plate. 

In [ ]:
x_offset = 1.5
y_offset = 0
openapi.move_to_well(openapi.labware_dct['6'], "A1", well_location="top", offset=(x_offset, y_offset, 1))

<Response [201]>

### Take solution off

In [35]:
# Take solution off the well palte with cuboids

# rows = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P']
columns = list(range(1,12+1))
rows = ['C']
z_offset = 1 # Offset in mm above the well bottom
takeoff_volume = 50 # Volume in ul to take off from the well
flow_rate  = 20 # Flow rate in ul/s
deck_slot_for_deposit = 3 # 6 well plate
deck_slot_for_origin = 6 # 96 well plate
well_for_dumping = "A2" # 6 well plate

row_idx = 0
column_idx = 0

while row_idx < len(rows):
    row = rows[row_idx]
    while column_idx < len(columns):
        column = columns[column_idx]
        r = openapi.aspirate(openapi.labware_dct[str(deck_slot_for_origin)], f"{row}{column}", well_location = 'bottom', offset = (x_offset, y_offset, z_offset), volume = takeoff_volume, flow_rate = flow_rate)
        responce_dict = json.loads(r.text)['data']
        if responce_dict['status'] == 'failed':
            if responce_dict['error']['errorType'] == 'InvalidAspirateVolumeError':
                print('Dumping fluid')
                openapi.blow_out(openapi.labware_dct[str(deck_slot_for_deposit)], well_for_dumping, well_location='center', flow_rate = 200)
        else:
            column_idx += 1
    column_idx = 0
    row_idx += 1

openapi.blow_out(openapi.labware_dct[str(deck_slot_for_deposit)], well_for_dumping, well_location='center', flow_rate = 200)
openapi.move_relative('z', 20)

Dumping fluid
Dumping fluid


<Response [201]>

# Wash tip

In [ ]:
rinse_location = 3 # Slot for the 6 well plate
well_to_wash = "A1" # Well where the cleaning solution is, like ethanol
well_to_rinse = "B1" # Well where the rinse solution is, like PBS
for i in range(3):
    openapi.aspirate(openapi.labware_dct[str(rinse_location)], well_to_wash, well_location='bottom', offset=(0,0,1), volume = 100, flow_rate = 200)
    openapi.dispense(openapi.labware_dct[str(rinse_location)], well_to_wash, well_location='bottom', offset=(0,0,1), volume = 100, flow_rate = 200)

for i in range(3):
    openapi.aspirate(openapi.labware_dct[str(rinse_location)], well_to_rinse, well_location='bottom', offset=(0,0,1), volume = 100, flow_rate = 200)
    openapi.dispense(openapi.labware_dct[str(rinse_location)], well_to_rinse, well_location='bottom', offset=(0,0,1), volume = 100, flow_rate = 200)

openapi.move_to_well(openapi.labware_dct[str(rinse_location)], well_to_rinse, well_location='top', offset=(0,0,10))

# Drop tip into bin

In [ ]:
openapi.move_to_coordinates((350, 345, 115)) # Moves the robot to hover above the trash bin
openapi.drop_tip_in_place()